<a href="https://colab.research.google.com/github/rubyvanrooyen/NIFTyworkshop/blob/master/Example_RMsynthesis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from ipywidgets import interact, widgets
from matplotlib import pylab
from scipy import signal
import math
import numpy

In [ ]:
def plot_complex(f, ax=None, show_legend=True):
    if ax is None:
        fig, ax = pylab.subplots(nrows=1, ncols=1, facecolor='white')
    ax.plot(numpy.absolute(f), 'g-', label='amp', lw=1)
    ax.plot(numpy.real(f), 'r-', label='real', lw=1)
    ax.plot(numpy.imag(f), 'b-.', label='imag', lw=1)

    if show_legend:
        ax.legend(loc=0)
    

In [ ]:
def p_lambda2(pl2):  # need better name (more descriptive)

    # define lambda^2
    dx = 3.835e-5 # m^2 
    x = numpy.arange((3.0e8/18e9)**2, (3.0e8/2e9)**2, dx)

    if 'slab' in pl2.lower():
        p = p0 * (numpy.sin(DRM * x )/(DRM*x) ) * numpy.exp(2j* RM * x) * numpy.exp(2j * PA)
    elif 'single' in pl2.lower():
        p = p0 * numpy.exp(2j * RM * x) * numpy.exp(2j * PA )
    elif 'two' in pl2.lower():
        p = p0 * numpy.exp(2j * RM * x) + p0 * numpy.exp(2j * RM1 *  x) * numpy.exp(2j * PA)

    return x, p


In [ ]:
#@title Parameters to use for the model
#@markdown select parameters and run to display simulated complex polarised intensity

#@markdown rotation measure, rad/m^2
RM = 0 #@param {type:"number"}
#@markdown gradient rad m^2
DRM = 500 #@param {type:"number"}
#@markdown position angle factor (pi/fp)
fp = 12.0 #@param {type:"number"}
PA = numpy.pi/fp # position angle
#@markdown ???
p0 = 0.5 #@param {type:"number"}
#@markdown ???
RM1 = 500 #@param {type:"number"}

#@markdown complex polarised surface brightness (???)
pl2 = "slab" #@param ["slab", "single component", "two component"]
x, p = p_lambda2(pl2)
plot_complex(p, show_legend=True)
pylab.show()

In [ ]:
def faraday_synthesis(x, pol, phi_range, lambda0=False):
    f = numpy.zeros(len(phi_range), dtype=complex)

    N = len(x)
    xmean = x.mean()

    for k, phi in enumerate(phi_range):
        if lambda0:
            f[k] = numpy.sum(pol * numpy.exp(-2j * phi * (x-xmean) ))/N
        else:
            f[k] = numpy.sum(pol * numpy.exp(-2j * phi * x ))/N
                    
    return f

In [ ]:
# define Faraday depth range
phi_range = numpy.arange(-10000, 10010, 10)

f = faraday_synthesis(x,  # wavelength: wavelength squared.
                      p,
                      phi_range,  # Faraday depth range
                      lambda0=False)

fig, axs = pylab.subplots(nrows=1, ncols=2, figsize=(17, 4), facecolor='white')
plot_complex(f, axs[0])
plot_complex(numpy.fft.fftshift(numpy.fft.fft(f)), axs[1])
pylab.show()

In [ ]:
def plot_rmclean(phi_range, F, rmtf, rmsf, clean_components):
    fig, ((ax1, ax2), (ax3, ax4)) = pylab.subplots(2, 2, figsize=(15, 15), facecolor='white')

    ax1.plot(phi_range, abs(F), 'g-', label='amp', lw=1)
    ax1.plot(phi_range, F.real, 'r-', label='real', lw=1)
    ax1.plot(phi_range, F.imag, 'b-', label='imag', lw=1)
    ax1.plot(phi_range, abs(rmsf), 'k-', label='amp', lw=1)
    ax1.set_xlabel('Faraday depth [rad m$^2$]')
    ax1.set_ylabel('Dirty')
    ax1.legend(loc='best')
    ax1.set_xlim(-1500, 1500)

    ax2.plot(phi_range, abs(rmsf), 'g-', label='amp', lw=1)
    ax2.plot(phi_range, rmsf.real, 'r-', label='real', lw=1)
    ax2.plot(phi_range, rmsf.imag, 'b-.', label='imag', lw=1)
    ax2.set_xlabel('Faraday depth [rad m$^2$]')
    ax2.set_ylabel('RMSF')
    ax2.legend(loc='best')
    ax2.set_xlim(-1500, 1500)

    ax3.plot(phi_range, abs(clean_components), 'g*', label='amp', lw=1)
    ax3.plot(phi_range, clean_components.real, 'r^', label='real', lw=1)
    ax3.plot(phi_range, clean_components.imag, 'bv', label='imag', lw=1)
    ax3.set_xlabel('Faraday depth [rad m$^2$]')
    ax3.set_ylabel('Model')
    ax3.legend(loc='best')
    ax3.set_xlim(-1500, 1500)

    ax4.plot(phi_range, abs(rmtf), 'g-', label='amp', lw=1)
    ax4.plot(phi_range, rmtf.real, 'r-', label='real', lw=1)
    ax4.plot(phi_range, rmtf.imag, 'b-.', label='imag', lw=1)
    ax4.set_xlabel('Faraday depth [rad m$^2$]')
    ax4.set_ylabel('RMTF')
    ax4.legend(loc='best')
    ax4.set_xlim(-1500, 1500)

In [ ]:
# width defines bar width
# percent defines current percentage
def progress(niter, width, percent):
    marks = math.floor(width * (percent / 100.0))
    spaces = math.floor(width - marks)

    loader = '[' + ('=' * int(marks)) + (' ' * int(spaces)) + ']'

    msg = "%d %s %d%% iterations used" % (niter, loader, percent)
    print(f'\r {msg}', end="", flush=True)

    if percent >= 100:
        print("\n")

In [ ]:
# rmclean(phi_range, f, x, gain=0.1, numiter=500, threshold=1e-3)
def rmclean(phi_range, F, wavelength, gain=0.1, 
            numiter=2, threshold=None, lambda0=False, verbose=False):


    """
    phi_range: Faraday depths. Lengths should be equal to pol.
    pol : the faraday spectrum (for single pol?)
    wavelength: wavelength squared.
    gain: the gain factor for cleaning.
    numiter: the number of iterations during looping.
    threshold:  not in use yet.
   
    Pseudo code:
    1. locate the maximum of pol.
       Store the index, Faraday depth max and amplitude max
    2. define a component = amplitude of max * gain
    3. Scale the RMTF (RMSF) by step 2, and shift to Faraday depth max
    4. Substract 3 from the original pol.
    5. Replace the original pol with the residual, and loop for numinter.
    6. Determine the ideal beam.
    7. Convolve the results from 5 with 6. Return this.
    """

    # create clean component list
    clean_components =  numpy.zeros([len(phi_range)], dtype=complex)

    # iterative identification of clean components
    for niter in range(numiter):
        clean_temp = numpy.zeros(len(phi_range), dtype=complex)
        # locate the maximum of pol.
        Famp = numpy.absolute(F)
        peak = Famp.max()
        if threshold is not None and peak < threshold:
            break
        # Store the index, Faraday depth max and amplitude max
        ind = numpy.where(Famp == peak)[0]
        phi_component = phi_range[ind[0]]  # Faraday depth

        if verbose:
            print(f"Iter {niter} component at {ind[0]}, phi {phi_component}, amp {peak}")
        else:
            progress(niter+1, 50, (float(niter+1)/float(numiter))*100)

        # define a component = amplitude of max * gain
        component =  F[ind[0]] * gain
        clean_temp[ind[0]] = component

        # Scale the RMTF (RMSF), and shift to Faraday depth max
        rmtf = faraday_synthesis(wavelength,
                                 numpy.ones(len(wavelength)),
                                 phi_range - phi_component,  # shift to Faraday depth max
                                 lambda0=lambda0)
        rmsf = component*rmtf  # scale rmsf

        if verbose:
            plot_rmclean(phi_range, F, rmtf, rmsf, clean_components)
            pylab.show()
            fig.clear()

        # Substract rmsf from the original pol.
        # Replace the original pol with the residual
        F = F - rmsf
        clean_components += clean_temp

    # Determine the ideal beam.
    Fresidual = F
    fwhm = (3.8/ (wavelength[0]-wavelength[-1]))
    sigma = (fwhm/2.35482)
    Gauss = numpy.exp(-0.5 * (phi_range/sigma)**2) 
 
    # Convolve the cleaned image with the ideal beam (add back the residual)
    fclean = signal.convolve(clean_components, Gauss, mode='same') + Fresidual
    return fclean, Fresidual, clean_components

#fclean, fres, fcomp = rmclean(phi_range, f, x, gain=0.1, numiter=0, threshold=1e-3, verbose=True)
#fclean, fres, fcomp = rmclean(phi_range, f, x, gain=0.1, numiter=1, threshold=1e-3, verbose=True)

In [ ]:
fclean, fres, fcomp = rmclean(phi_range, f, x, gain=0.1, numiter=500, threshold=1e-3)

In [ ]:
flambda0 = faraday_synthesis(x, p, phi_range, lambda0=True)
fcleanlambda0, fres, fcomp = rmclean(phi_range, flambda0, x, gain=0.1, numiter=500, lambda0=True, threshold=1e-3)

In [ ]:
fig, ((ax1, ax2), (ax3, ax4)) = pylab.subplots(2, 2, figsize=(15, 15), facecolor='white')

ax1.set_title('No $\lambda_0$')
ax1.plot(phi_range, numpy.absolute(f), 'k-', label='amp', lw=1)
ax1.plot(phi_range, f.real, 'k--', label='real', lw=1)
ax1.plot(phi_range, f.imag, 'k-.', label='imag', lw=1)
ax1.set_xlabel('Faraday depth [rad m$^2$]')
ax1.set_ylabel('Faraday Spectrum')
ax1.legend(loc='best')
ax1.set_xlim(-1500, 1500)

ax3.plot(phi_range, numpy.absolute(fclean), 'k-', label='amp', lw=1)
ax3.plot(phi_range, fclean.real, 'k--', label='real', lw=1)
ax3.plot(phi_range, fclean.imag, 'k-.', label='imag', lw=1)
ax3.set_xlabel('Faraday depth [rad m$^2$]')
ax3.set_ylabel('Faraday Spectrum')
ax3.legend(loc='best')
ax3.set_xlim(-1500, 1500)


ax2.set_title('With $\lambda_0$')
ax2.plot(phi_range, numpy.absolute(flambda0), 'k-', label='amp', lw=1)
ax2.plot(phi_range, flambda0.real, 'k--', label='real', lw=1)
ax2.plot(phi_range, flambda0.imag, 'k-.', label='imag', lw=1)
ax2.set_xlabel('Faraday depth [rad m$^2$]')
ax2.set_ylabel('Faraday Spectrum')
ax2.legend(loc='best')
ax2.set_xlim(-1500, 1500)

ax4.plot(phi_range, numpy.absolute(fcleanlambda0), 'k-', label='amp', lw=1)
ax4.plot(phi_range, fcleanlambda0.real, 'k--', label='real', lw=1)
ax4.plot(phi_range, fcleanlambda0.imag, 'k-.', label='imag', lw=1)
ax4.set_xlabel('Faraday depth [rad m$^2$]')
ax4.set_ylabel('Faraday Spectrum')
ax4.legend(loc='best')
ax4.set_xlim(-1500, 1500)

pylab.show()